In [1]:
!{'pip install --quiet apache_beam'}

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 8.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.5/17.5 MB 69.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 78.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 96.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 58.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.5/261.5 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31

In [12]:
import apache_beam as beam

class PairPerson(beam.DoFn):
  def process(self, element, dept):
    return [(dept+','+element[1], 1)]

def filter_on_count(element):
  name, count = element
  if count> 30:
    return element

def format_output(element):
  name, count = element
  return ','.join((name, str(count), 'Regular employee'))

class MyTransform(beam.PTransform):

  def expand(self, input_collection):
    a = (
        input_collection
        | 'Group and sum' >> beam.CombinePerKey(sum)
        | 'count filter' >> beam.Filter(filter_on_count)
        | 'Regular accounts employee'>> beam.Map(format_output)
    )
    return a

with beam.Pipeline() as p:
  input_collection = (
      p
      | 'read from text file' >> beam.io.ReadFromText('drive/MyDrive/APACHE_BEAM/datasets/dept_data.txt')
      | 'split_row' >> beam.Map(lambda record: record.split(","))
  )

  accounts_count = (
      input_collection
      | 'filter Accounts dept persons' >> beam.Filter(lambda record: record[3]=="Accounts")
      | 'pair account with 1' >> beam.ParDo(PairPerson(), 'Accounts')
      | 'composite account'>> MyTransform() # calling composite transform class here
  )

  hr_count = (
      input_collection
      | 'filter hr dept persons' >> beam.Filter(lambda record: record[3]=="HR")
      | 'pair hr with 1' >> beam.ParDo(PairPerson(), 'HR')
      | 'composite hr'>> MyTransform() # calling composite transform class here
  )

  combine_count = (
      (accounts_count, hr_count)
      | beam.Flatten()
      | beam.Map(print)
  )



Accounts,Marco,31,Regular employee
Accounts,Rebekah,31,Regular employee
Accounts,Itoe,31,Regular employee
Accounts,Edouard,31,Regular employee
Accounts,Kyle,62,Regular employee
Accounts,Kumiko,31,Regular employee
Accounts,Gaston,31,Regular employee
HR,Beryl,62,Regular employee
HR,Olga,31,Regular employee
HR,Leslie,31,Regular employee
HR,Mindy,31,Regular employee
HR,Vicky,31,Regular employee
HR,Richard,31,Regular employee
HR,Kirk,31,Regular employee
HR,Kaori,31,Regular employee
HR,Oscar,31,Regular employee
